In [3]:
import LaTeXPy as l

In [6]:
l.l(r"""
$5+5$
""")


$
\renewcommand{\And}{\ \text{and}\ }
\newcommand{\Or}{\ \text{or}\ }
\newcommand{\Not}{\text{not}\ }
\newcommand{\m}{\mathbf}
\newcommand{\bb}{\mathbb}
\newcommand{\cc}{\mathcal}
\newcommand{\s}{\text}
\newcommand{\bsl}{\backslash}
\newcommand{\sm}{{\sim}}
\newcommand{\tup}[1]{(#1)}
\newcommand{\Mod}{\text{Mod}}
\newcommand{\Con}{\text{Con}}
\newcommand{\Pre}{\text{Pre}}
5+5$
